# Getting the distant pairs in a set

First, import numpy and pandas for generating and manipulating the data.

In [1]:
import numpy as np
import pandas as pd

Next, generate a random sample of 7 integers between 100000 and 999999 as `test`.

In [46]:
test = np.random.randint(100000, 999999, size=7)
test

array([713435, 441293, 560063, 791195, 529616, 143613, 486992])

The variable `test` represents a one-dimensional set of values where we want to find the maximum distance between two values in the set, as well as which two values determine the maximum distance.

This can be done in three steps:
* Take the cartesian product of the set with itself and apply the distance algorithm to every pair.
* Find the maximum value in the set of distances.
* Determine which pairs in the set produce the maximum distance.

First, the cartesian product of the set with itself can be represented as a pandas DataFrame, where the index and column headers of the DataFrame are the values in the set.

In [47]:
df = pd.DataFrame(index=test, columns=test)
df

,713435,441293,560063,791195,529616,143613,486992
713435,NaN,NaN,NaN,NaN,NaN,NaN,NaN
441293,NaN,NaN,NaN,NaN,NaN,NaN,NaN
560063,NaN,NaN,NaN,NaN,NaN,NaN,NaN
791195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529616,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143613,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486992,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This creates a n x n DataFrame, where n is the number of values in the set. Since this DataFrame we created without data, all the values are empty.

The next set is to apply the distance algorithm which is simply `abs(value1 - value2)` where `value1` and `value2` are both in the set. Applying this algorithm to every cell in the DataFrame is equivalent to calculating the distance between every value in the original set.

This is accomplished by using the DataFrame object's `apply()` method, where the argument is a callable function. Since the distance algorithm is simple, a lambda function will suffice. Using `apply()` is always faster than using nested `for` loops because `apply()` is a vectorized operation applied to every value in a row simultaneously. In essence, `apply()` is a O(n) time operation where n is the number of rows and nested `for` loops is a O(n^2) time operation.

In [55]:
diff = df.apply(lambda x: abs(x.name - x.index))
diff

,713435,441293,560063,791195,529616,143613,486992
713435,0,272142,153372,77760,183819,569822,226443
441293,272142,0,118770,349902,88323,297680,45699
560063,153372,118770,0,231132,30447,416450,73071
791195,77760,349902,231132,0,261579,647582,304203
529616,183819,88323,30447,261579,0,386003,42624
143613,569822,297680,416450,647582,386003,0,343379
486992,226443,45699,73071,304203,42624,343379,0


What did this do? The `apply()` method performs the lambda function on each row and subtracts the name of the row (the index value of the DataFrame) from the index of the row (the column header of the DataFrame).

This populates the DataFrame with the absolute value distance between each pair in the original set. Notice how all the values along the diagonal are 0, because a number subtracted from itself is 0. This also indicates that the DataFrame is partitioned into two sections, each a triangle that mirror's the other. This means the max distance is in the DataFrame at least twice.

The next step is to find the max distance. This is done by concatenating the columns of the DataFrame together into a single numpy array, then taking the maximum value in the array.

In [51]:
flat = pd.concat([diff[col] for col in diff.columns], ignore_index=True)
flat

0          0
1     272142
2     153372
3      77760
4     183819
5     569822
6     226443
7     272142
8          0
9     118770
10    349902
11     88323
12    297680
13     45699
14    153372
15    118770
16         0
17    231132
18     30447
19    416450
20     73071
21     77760
22    349902
23    231132
24         0
25    261579
26    647582
27    304203
28    183819
29     88323
30     30447
31    261579
32         0
33    386003
34     42624
35    569822
36    297680
37    416450
38    647582
39    386003
40         0
41    343379
42    226443
43     45699
44     73071
45    304203
46     42624
47    343379
48         0
dtype: int64

In [52]:
max = flat.max()
max

647582

Now that the maximum distance is found, the last step is to determine which two pairs in the original set form the max distance. Since the index and column headers of the DataFrame are the values in the original set, the DataFrame can be filtered for just the max distance value.

Since the max distance value appears in the DataFrame at least twice, filtering the DataFrame will lead to a resulting 2 x 2 DataFrame where both values in the index and column headers are the distant pairs.

In [53]:
filtered = diff[diff == max].dropna(how='all').dropna(how='all', axis=1)
filtered

,791195,143613
791195,NaN,647582.0
143613,647582.0,NaN


In [54]:
pairs = filtered.index[0], filtered.index[1]
pairs

(791195, 143613)